## 스타벅스 매장정보 
https://www.starbucks.co.kr/store/store_map.do

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import time

def get_sido_list():
    url = "https://www.starbucks.co.kr/store/getSidoList.do"
    response = requests.post(url)
    data = json.loads(response.text)
    return {item['sido_cd']: item['sido_nm'] for item in data['list'] if item['sido_cd'] and item['sido_nm']}

def get_stores(sido_cd):
    url = "https://www.starbucks.co.kr/store/getStore.do"
    payload = {
        "r": "PWQMWVRA5Y",
        "in_biz_cds": "0",
        "in_scodes": "0",
        "ins_lat": "37.56682",
        "ins_lng": "126.97865",
        "search_text": "",
        "p_sido_cd": sido_cd,
        "p_gugun_cd": "",
        "isError": "true",
        "in_distance": "0",
        "in_biz_cd": "",
        "iend": "1000",
        "searchType": "C",
        "set_date": "",
        "rndCod": "AKITO9STMQ",
        "all_store": "0"
    }
    response = requests.post(url, data=payload)
    return json.loads(response.text)['list']

In [ ]:
sido_list = get_sido_list()

In [ ]:
all_stores = []  # 모든 매장 데이터를 저장할 리스트

for sido_cd, sido_nm in sido_list.items():
    print(f"Collecting stores in {sido_nm}...")
    stores = get_stores(sido_cd)
    print(f"수집된 매장 수: {len(stores)}")
    all_stores.extend(stores)  # 리스트에 매장 데이터 추가
    time.sleep(1)  # 서버 부하를 줄이기 위한 대기 시간

In [ ]:
# DataFrame 생성 및 전처리
df = pd.DataFrame(all_stores)
# df = df.replace([0, ""], np.nan).dropna(axis=1, how="all")
df = df[["sido_name","gugun_name","s_name","addr","lat","lot"]]
df

In [ ]:
df.info()

In [ ]:
# CSV 파일 저장
df.to_csv('starbucks_stores.csv', index=False)
print("\nCSV 파일 저장 완료: starbucks_stores.csv")

# 데이터 확인
print(f"\n총 매장 수: {len(df)}")
print(f"\n지역별 매장 수:\n{df['sido_name'].value_counts()}")